In [36]:
import numpy
from collections import defaultdict
from math import ceil, floor, comb

In [11]:
wep = { 'attacks': '3', 'bsws': 3, 'strength': 5, 'AP': -2, 'damage': '2', 'lh':True, 'dw':True }
target = { 'toughness': 3, 'save': 2, 'invuln': 4, 'wounds': 3, 'fnp': 5 }

In [ ]:
def dam_prob(wep,target,crit_hit=None):

    # TODO:
    # Cover / Ignores cover

    # Prob to hit
    if wep.get('torrent'):
        p_hcrit, p_hit = 0, 1
    else:
        hit_crit = 6 # crit on 6, but can be on 5+
        p_hcrit = (7-hit_crit)/6.0
        p_hit = max((6*p_hcrit),min(5, # hit_crit always hits, 1 always misses
                    (7-wep['bsws'])))/6.0
        
    # Check if fn parameter already tells us if it was a crit or not
    # This behavior is needed for Sustained hits as they also affect hit counts
    if crit_hit: p_hcrit,p_hit = 1,1
    elif crit_hit==False: p_hcrit,p_hit = 0, p_hit-p_hcrit

    print("Hit",p_hit,p_hcrit)

    # Prob to wound

    wound_crit = 6 # Anti-keywords change this
    p_wcrit = (7-wound_crit)/6.0

    if wep['strength']>target['toughness']:
        if wep['strength']>=2*target['toughness']:
            p_wound = 5/6.0
        else: p_wound = 4/6.0
    elif wep['strength']<target['toughness']:
        if 2*wep['strength']<=target['toughness']:
            p_wound = 1/6.0
        else: p_wound = 2/6.0
    else: p_wound = 3/6.0

    p_wound = max(p_wcrit,p_wound)
    
    print("Wound",p_wound,p_wcrit)

    # Prob to not save
    save = min(target['invuln'],target['save']-wep['AP'])
    p_nsave = 1.0 - max(1,min(6,7-save))/6.0

    if wep.get('dw'): # Devastating wounds
        p_nsave = ((p_wound-p_wcrit)*p_nsave + p_wcrit)/p_wound

    print("Save",p_nsave)

    # Total probability
    if wep.get('lh'):
        p_dam = p_hcrit*p_nsave + (p_hit-p_hcrit)*p_wound*p_nsave
    else:
        p_dam = p_hit*p_wound*p_nsave

    print("Total prob", p_dam)

    return p_dam

In [54]:
# Helper functions for damage distributions

def convolve(d1,d2):
    res = defaultdict(lambda: 0)
    for k1,v1 in d1.items():
        for k2,v2 in d2.items():
            res[k1+k2] += v1*v2 
    return res

def mult_ddist_vals(d, val):
    res = defaultdict(lambda: 0)
    for k1,v1 in d.items():
        res[int(ceil(k1*val))] += v1 
    return res

def mult_ddist_probs(d,p):
    return { k:v*p for k,v in d.items() }

def threshold_ddist(dd,val,lt=True):
    for k in list(dd.keys()): 
        if (lt and k<val) or (not lt and k>val): 
            dd[val]+=dd[k]
            del dd[k]
    
def flatdist(n):
    return { (i+1):1/n for i in range(n) }

# q is the probability of saving the damage
def fnp_transform(d, q):
    p = 1.0-q
    res = defaultdict(lambda: 0)
    for k,v in d.items():
        for i in range(k+1): # Binomial distribution
            res[i] += v*comb(k,i)*(p**i)*(q**(k-i))
    return res

def dd_mean(dd):
    val = 0.0
    for k, v in dd.items():
        val += k*v
    return val


In [70]:
def dam_dist(dmgstr, n_wounds=None, div=1,mult=1,add=0,fnp=None):

    # Create dmgstr distribution
    dd = { 0: 1.0 }
    for t in dmgstr.split('+'):
        if 'D' in t:
            d = t.split('D')
            if d[0]=='': d = [1,int(d[1])]
            else: d = list(map(int,d))
            for _ in range(d[0]):
                nd = flatdist(d[1])
                dd = convolve(dd,nd)
        else:
            d = int(t)
            dd = convolve(dd,{d:1.0})

    # Apply simple modifiers
    if div!=1: dd = mult_ddist_vals(dd,1.0/div)
    if mult!=1: dd = mult_ddist_probs(dd,mult)
    if add!=0: dd = convolve(dd,{add:1.0})

    # Threshold to 1
    threshold_ddist(dd,1,True)

    # Apply FNP
    if fnp: dd = fnp_transform(dd,(7-fnp)/6)

    # Threshold to n_wounds
    if n_wounds: threshold_ddist(dd,n_wounds,False)

    print(dd_mean(dd))

    return dd
    
dam_dist('2D6+2',n_wounds=2,div=2, add=-2,fnp=4)#None)

1.2178819444444444


defaultdict(<function __main__.fnp_transform.<locals>.<lambda>()>,
            {0: 0.19878472222222224,
             1: 0.3845486111111111,
             2: 0.4166666666666667})